# Installations and Imports

In [1]:
# Installations of libraries
!pip install PyPDF2

In [2]:
!pip install nltk

In [3]:
!pip install pdfminer

In [4]:
!pip install textblob

In [5]:
!pip install spacy

In [6]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-03-16 19:00:56.288784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-16 19:00:56.289767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
# Imports
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from textblob import TextBlob
from collections import Counter
from spacy import displacy

import spacy
import en_core_web_sm
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

import io
import nltk

In [10]:
# nltk download
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# PDF Article to Text

In [11]:
# Initiate pdf file
pdf_file = "../res/who_int_0.pdf"
# Initiate dump file
pdf_to_text = "../dump/pdf_to_text.txt"
df_to_text = "../dump/df_to_text.txt"

In [12]:
# https://stackoverflow.com/questions/64467930/python-pypdf2-extracttext-not-working
inFile = open(pdf_file, 'rb')
resMgr = PDFResourceManager()
retData = io.StringIO()
TxtConverter = TextConverter(resMgr, retData,laparams = LAParams())
interpreter = PDFPageInterpreter(resMgr, TxtConverter)

# Process each pages
for page in PDFPage.get_pages(inFile): 
    interpreter.process_page(page)
    

# Write to text temporary file
txt = retData.getvalue()

In [13]:
# Acquire sentences
blob = TextBlob(txt)
sentences = []

# Append to sentences whilest removing 'sentence(...)' on each sentences/list elements
for s in blob.sentences:
     sentences.append(str(s).strip()),

cleaned_sentences = []

# Append to cleaned_sentences whilest removing new lines or '\n'
for x in sentences:
     cleaned_sentences.append(x.replace("\n", " "))

# Sample data inside cleaned_sentences
cleaned_sentences[0:2]

['3/15/22, 10:32 AM  Is the pandemic ending soon?',
 'Western Pacific Philippines  ©  Opinion: Is the pandemic ending soon?']

In [14]:
# Create and Open new temp text file and write cleaned sentences on a per line basis
with open(pdf_to_text, 'w') as f: 
     f.write('\n'.join(cleaned_sentences))

# Text to DataFrame

In [15]:
# Open text file
raw = open(pdf_to_text).read()

In [17]:
nlp = en_core_web_sm.load()
raw_nlp = nlp(raw)

In [18]:
sentences = [x for x in raw_nlp.sents]

In [19]:
sentence_index = []

# Contain tokenized data and its corresponding sentence index
for idx1 in range(len(sentences)):
    temporary_sentence = sentences[idx1]
    temporary_tokens = [x for x in temporary_sentence]
    
    for idx2, val in enumerate(temporary_tokens):
        sentence_index.append(idx1)

print(sentence_index[0:20])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [20]:
ent_type = ([(x, x.pos_, spacy.explain(x.tag_), x.tag_, x.ent_iob_, x.ent_type_) for x in raw_nlp])

# Words and Tags list
word, pos, e_tags, tags, iob, e_type = zip(*ent_type)

In [21]:
# Length must be the same
print(len(sentence_index), type(sentence_index))
print(len(word), type(word))
print(len(pos), type(pos))
print(len(e_tags), type(e_tags))
print(len(tags), type(tags))
print(len(iob), type(iob))
print(len(e_type), type(e_type))

1464 <class 'list'>
1464 <class 'tuple'>
1464 <class 'tuple'>
1464 <class 'tuple'>
1464 <class 'tuple'>
1464 <class 'tuple'>
1464 <class 'tuple'>


In [22]:
df = pd.DataFrame(zip(sentence_index, word, pos, e_tags, tags, iob, e_type), 
                  columns=["Sentence_Index", "Token", "Pos", "Explained_Tag", "Tag", "iob_Tag", "Entity_Type"])
df

,Sentence_Index,Token,Pos,Explained_Tag,Tag,iob_Tag,Entity_Type
0,0,3/15/22,NUM,cardinal number,CD,B,CARDINAL
1,0,",",PUNCT,"punctuation mark, comma",",",O,
2,0,10:32,NUM,cardinal number,CD,B,TIME
3,0,AM,NOUN,"noun, singular or mass",NN,I,TIME
4,0,,SPACE,whitespace,_SP,O,
...,...,...,...,...,...,...,...
1459,81,.,PUNCT,"punctuation mark, sentence closer",.,O,
1460,82,\n,SPACE,whitespace,_SP,O,
1461,82,https://www.who.int/philippines/news/detail/11...,PROPN,"noun, proper singular",NNP,O,
1462,82,,SPACE,whitespace,_SP,O,


In [23]:
# Output to csv
df.to_csv('../data/csv_dataset/data.csv', index=True, header=True)

# DataFrame to Text

In [24]:
nlp = en_core_web_sm.load()

text = nlp(raw)

sentences = [x.text for x in text.sents]
print(sentences[0])

3/15/22, 10:32 AM  Is the pandemic ending soon?


In [25]:
labels = [x.label_ for x in text.ents]
Counter(labels)

Counter({'CARDINAL': 9,
         'TIME': 3,
         'ORG': 16,
         'DATE': 6,
         'PERSON': 4,
         'GPE': 10,
         'PERCENT': 1,
         'PRODUCT': 2,
         'LOC': 2,
         'NORP': 1,
         'WORK_OF_ART': 1})

In [26]:
displacy.render(nlp(str(sentences[0])), jupyter=True, style='ent')

In [27]:
print(len(labels))

55
